In [1]:
import pandas as pd
data_aim = pd.read_csv('tmpcomments.csv')['comment_text']
data_just = pd.read_csv('comments.csv')['comment_text']

In [2]:
import numpy as np

In [8]:

data = pd.concat([pd.DataFrame({'comment' : data_just[:200],'label':pd.Series(np.zeros(200))}),pd.DataFrame({'comment' : data_aim,'label':pd.Series(np.ones(len(data_aim)))})])

In [9]:
from sklearn.model_selection import train_test_split
X = data['comment']  
y = data['label']    


X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,      
    stratify=y,         
    random_state=42
)


train_data = pd.DataFrame({'comment': X_train, 'label': y_train})
val_data = pd.DataFrame({'comment': X_val, 'label': y_val})


train_data.to_csv('train_comments.csv', index=False)
val_data.to_csv('val_comments.csv', index=False)

In [12]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer
from DataSet import CommentDataset
from sklearn.metrics import accuracy_score, f1_score
train_dataset = CommentDataset("train_comments.csv")
eval_dataset = CommentDataset("val_comments.csv")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}


model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=16,
    num_train_epochs=5,

)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\JACK\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.358540,0.935897,0.935866
2,No log,0.194478,0.948718,0.948650
3,No log,0.142759,0.961538,0.961443
4,No log,0.139789,0.961538,0.961443
5,No log,0.139500,0.961538,0.961443


TrainOutput(global_step=100, training_loss=0.22680383682250976, metrics={'train_runtime': 60.5648, 'train_samples_per_second': 25.427, 'train_steps_per_second': 1.651, 'total_flos': 50999948482560.0, 'train_loss': 0.22680383682250976, 'epoch': 5.0})

In [32]:
save_directory = "./bert_classifier"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Модель и токенизатор сохранены в {save_directory}")

Модель и токенизатор сохранены в ./bert_classifier
